## Read data

In [1]:
# China

In [1]:
import pandas as pd
def file_reader(path):
    df=pd.DataFrame()
    chunksize = 10 ** 6
    for chunk in pd.read_csv(path, chunksize=chunksize, index_col=[0]):
        df = df.append(chunk)
    return df

country='China'
data=file_reader('/Volumes/BECCA/Project/Data/Input/02_Preprocessing/05_Final/%s.csv' %country)

In [2]:
data

,tweet_text,polarity_number,subjectivity_number,repeat_tag,freq_tag,day,hour,minute,suspend
0,Necessity is the mother of invention,0.500000,0.000000,0,0,4,21,21,1
1,#病毒 Constantly defying the will of the citize...,0.425000,0.316667,1,0,3,5,9,1
2,#肺炎 #病毒 The priceless life should not be the...,0.654167,0.383333,0,0,5,2,37,1
3,"Want to see the walking dead, but feel a littl...",0.403125,0.450000,0,0,2,5,57,1
4,Maybe should call that Kekong? #HongKong #Keki...,0.500000,0.000000,0,0,4,1,39,1
...,...,...,...,...,...,...,...,...,...
44293,@ace_critic @realDonaldTrump @narendramodi The...,0.500000,0.000000,0,0,3,2,48,0
44294,RT @F1Bathurst2015: @catherine___c @tyrelle123...,0.538889,0.544444,0,0,3,9,24,0
44295,RT @YeOldHongKonger: .\nBioterorist Xi Jinping...,0.500000,0.100000,0,0,3,0,25,0
44296,@GauravS101418 @narendramodi Hahaha... So u ex...,0.600000,0.400000,0,0,3,1,57,0


## Feature selection

In [4]:
from sklearn import preprocessing
import numpy as np
min_max_scaler = preprocessing.MinMaxScaler()
x=np.array(data['polarity_number']).reshape(-1,1)
data.polarity_number=min_max_scaler.fit_transform(x)
data.suspend=data['suspend'].astype('int64')
data.repeat_tag=data['repeat_tag'].astype('int64')
data.freq_tag=data['freq_tag'].astype('int64')
data

,tweet_text,polarity_number,subjectivity_number,repeat_tag,freq_tag,day,hour,minute,suspend
0,Necessity is the mother of invention,0.500000,0.000000,0,0,4,21,21,1
1,#病毒 Constantly defying the will of the citize...,0.425000,0.316667,1,0,3,5,9,1
2,#肺炎 #病毒 The priceless life should not be the...,0.654167,0.383333,0,0,5,2,37,1
3,"Want to see the walking dead, but feel a littl...",0.403125,0.450000,0,0,2,5,57,1
4,Maybe should call that Kekong? #HongKong #Keki...,0.500000,0.000000,0,0,4,1,39,1
...,...,...,...,...,...,...,...,...,...
44293,@ace_critic @realDonaldTrump @narendramodi The...,0.500000,0.000000,0,0,3,2,48,0
44294,RT @F1Bathurst2015: @catherine___c @tyrelle123...,0.538889,0.544444,0,0,3,9,24,0
44295,RT @YeOldHongKonger: .\nBioterorist Xi Jinping...,0.500000,0.100000,0,0,3,0,25,0
44296,@GauravS101418 @narendramodi Hahaha... So u ex...,0.600000,0.400000,0,0,3,1,57,0


In [5]:
from sklearn.feature_selection import VarianceThreshold
import numpy as np
X = np.array(data)[:,1:8].astype(float)
sel = VarianceThreshold()
sel = sel.fit_transform(X)

In [7]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
y = np.array(data)[:,8].astype(float)
X_new = SelectKBest(chi2, k=2).fit_transform(X, y)
X_new
# 2: freq_tag & hour
# 3: repeated_tag & freq_tag & hour
# 4: day & repeated_tag & freq_tag & hour
# 5: minute & day & repeated_tag & freq_tag & hour
# 6: subjectivity_number & minute & day & repeated_tag & freq_tag & hour

array([[ 0., 21.],
       [ 1.,  5.],
       [ 0.,  2.],
       ...,
       [ 0.,  0.],
       [ 0.,  1.],
       [ 0., 16.]])

In [7]:
#L1-based feature selection

from sklearn.svm import LinearSVC
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectFromModel

lsvc = LinearSVC(C=0.01, penalty="l1", dual=False).fit(X, y)
model = SelectFromModel(lsvc, prefit=True)
X_new = model.transform(X)
X_new.shape

(44294, 7)

In [8]:
#Tree-based feature selection

from sklearn.ensemble import ExtraTreesClassifier
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectFromModel

X.shape

(44294, 7)

In [9]:
clf = ExtraTreesClassifier(n_estimators=50)
clf = clf.fit(X, y)
clf.feature_importances_  

array([0.1560871 , 0.16606408, 0.03494924, 0.03999021, 0.09272968,
       0.28642631, 0.22375338])

In [10]:
model = SelectFromModel(clf, prefit=True)
X_new = model.transform(X)
X_new

array([[ 0.5       ,  0.        , 21.        , 21.        ],
       [ 0.425     ,  0.31666667,  5.        ,  9.        ],
       [ 0.65416667,  0.38333333,  2.        , 37.        ],
       ...,
       [ 0.5       ,  0.1       ,  0.        , 25.        ],
       [ 0.6       ,  0.4       ,  1.        , 57.        ],
       [ 0.53472222,  0.36944444, 16.        , 58.        ]])

In [11]:
data

,tweet_text,polarity_number,subjectivity_number,repeat_tag,freq_tag,day,hour,minute,suspend
0,Necessity is the mother of invention,0.500000,0.000000,0,0,4,21,21,1
1,#病毒 Constantly defying the will of the citize...,0.425000,0.316667,1,0,3,5,9,1
2,#肺炎 #病毒 The priceless life should not be the...,0.654167,0.383333,0,0,5,2,37,1
3,"Want to see the walking dead, but feel a littl...",0.403125,0.450000,0,0,2,5,57,1
4,Maybe should call that Kekong? #HongKong #Keki...,0.500000,0.000000,0,0,4,1,39,1
...,...,...,...,...,...,...,...,...,...
44293,@ace_critic @realDonaldTrump @narendramodi The...,0.500000,0.000000,0,0,3,2,48,0
44294,RT @F1Bathurst2015: @catherine___c @tyrelle123...,0.538889,0.544444,0,0,3,9,24,0
44295,RT @YeOldHongKonger: .\nBioterorist Xi Jinping...,0.500000,0.100000,0,0,3,0,25,0
44296,@GauravS101418 @narendramodi Hahaha... So u ex...,0.600000,0.400000,0,0,3,1,57,0


In [12]:
data.to_csv('/Volumes/BECCA/Project/Data/Input/02_Preprocessing/05_Final/%s.csv' %country)

In [13]:
# Russia

In [9]:
import pandas as pd
def file_reader(path):
    df=pd.DataFrame()
    chunksize = 10 ** 6
    for chunk in pd.read_csv(path, chunksize=chunksize, index_col=[0]):
        df = df.append(chunk)
    return df

country='Russia'
data=file_reader('/Volumes/BECCA/Project/Data/Input/02_Preprocessing/05_Final/%s.csv' %country)

In [10]:
data

,tweet_text,polarity_number,subjectivity_number,repeat_tag,day,hour,minute,suspend
0,"Barcelona is so sunny and nice at Vmworld, tre...",0.700000,0.600000,0,6,10,32,1
1,Linear -up and –out scaling on disk and node l...,0.600000,0.200000,0,1,12,30,1
2,open for a surprise https://t.co/YrU8x3AhGC,0.500000,0.500000,0,5,11,27,1
3,MY HEART 😭❤️ https://t.co/iKrYLK5apF,0.500000,0.000000,0,1,12,21,1
4,cows are just big grass puppies https://t.co/6...,0.466667,0.133333,0,4,20,26,1
...,...,...,...,...,...,...,...,...
54165,RT @spacebongmilk: My girl can be butt ass nak...,0.500000,0.233333,0,2,18,19,0
54166,"Not a win again, but people that understands f...",0.600000,0.400000,0,3,2,29,0
54167,RT @alizawhoo: @lanascouture A girl can walk o...,0.600000,0.400000,0,2,16,56,0
54168,People don't understand what football means to...,0.396875,0.443750,0,3,7,4,0


## Feature selection

In [11]:
from sklearn import preprocessing
import numpy as np
min_max_scaler = preprocessing.MinMaxScaler()
x=np.array(data['polarity_number']).reshape(-1,1)
data.polarity_number=min_max_scaler.fit_transform(x)
data.suspend=data['suspend'].astype('int64')
data.repeat_tag=data['repeat_tag'].astype('int64')
# data.freq_tag=data['freq_tag'].astype('int64')
data

,tweet_text,polarity_number,subjectivity_number,repeat_tag,day,hour,minute,suspend
0,"Barcelona is so sunny and nice at Vmworld, tre...",0.700000,0.600000,0,6,10,32,1
1,Linear -up and –out scaling on disk and node l...,0.600000,0.200000,0,1,12,30,1
2,open for a surprise https://t.co/YrU8x3AhGC,0.500000,0.500000,0,5,11,27,1
3,MY HEART 😭❤️ https://t.co/iKrYLK5apF,0.500000,0.000000,0,1,12,21,1
4,cows are just big grass puppies https://t.co/6...,0.466667,0.133333,0,4,20,26,1
...,...,...,...,...,...,...,...,...
54165,RT @spacebongmilk: My girl can be butt ass nak...,0.500000,0.233333,0,2,18,19,0
54166,"Not a win again, but people that understands f...",0.600000,0.400000,0,3,2,29,0
54167,RT @alizawhoo: @lanascouture A girl can walk o...,0.600000,0.400000,0,2,16,56,0
54168,People don't understand what football means to...,0.396875,0.443750,0,3,7,4,0


In [12]:
from sklearn.feature_selection import VarianceThreshold
import numpy as np
X = np.array(data)[:,1:7].astype(float)
sel = VarianceThreshold()
sel = sel.fit_transform(X)

In [13]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
y = np.array(data)[:,7].astype(float)
X_new = SelectKBest(chi2, k=1).fit_transform(X, y)
X_new

array([[10.],
       [12.],
       [11.],
       ...,
       [16.],
       [ 7.],
       [22.]])

In [19]:
#L1-based feature selection

from sklearn.svm import LinearSVC
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectFromModel

lsvc = LinearSVC(C=0.01, penalty="l1", dual=False).fit(X, y)
model = SelectFromModel(lsvc, prefit=True)
X_new = model.transform(X)
X_new.shape

(54164, 5)

In [20]:
#Tree-based feature selection

from sklearn.ensemble import ExtraTreesClassifier
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectFromModel

X.shape

(54164, 6)

In [21]:
clf = ExtraTreesClassifier(n_estimators=50)
clf = clf.fit(X, y)
clf.feature_importances_  

array([0.16653939, 0.22485762, 0.00065475, 0.15185861, 0.26840924,
       0.18768038])

In [22]:
model = SelectFromModel(clf, prefit=True)
X_new = model.transform(X)
X_new

array([[ 0.6    , 10.     , 32.     ],
       [ 0.2    , 12.     , 30.     ],
       [ 0.5    , 11.     , 27.     ],
       ...,
       [ 0.4    , 16.     , 56.     ],
       [ 0.44375,  7.     ,  4.     ],
       [ 0.825  , 22.     , 45.     ]])

In [23]:
data

,tweet_text,polarity_number,subjectivity_number,repeat_tag,day,hour,minute,suspend
0,"Barcelona is so sunny and nice at Vmworld, tre...",0.700000,0.600000,0,6,10,32,1
1,Linear -up and –out scaling on disk and node l...,0.600000,0.200000,0,1,12,30,1
2,open for a surprise https://t.co/YrU8x3AhGC,0.500000,0.500000,0,5,11,27,1
3,MY HEART 😭❤️ https://t.co/iKrYLK5apF,0.500000,0.000000,0,1,12,21,1
4,cows are just big grass puppies https://t.co/6...,0.466667,0.133333,0,4,20,26,1
...,...,...,...,...,...,...,...,...
54165,RT @spacebongmilk: My girl can be butt ass nak...,0.500000,0.233333,0,2,18,19,0
54166,"Not a win again, but people that understands f...",0.600000,0.400000,0,3,2,29,0
54167,RT @alizawhoo: @lanascouture A girl can walk o...,0.600000,0.400000,0,2,16,56,0
54168,People don't understand what football means to...,0.396875,0.443750,0,3,7,4,0


In [24]:
data.to_csv('/Volumes/BECCA/Project/Data/Input/02_Preprocessing/05_Final/%s.csv' %country)